In [1]:
from __future__ import absolute_import, division, print_function

# Elevation & Length of Day

To account for the amount of sunlight avaialable to the plants at the different locations and times, get the length of the day. Here, I do this by using the 'Astra' package.

I also leverage the Google Elevation API to retrieve the elevation for each location.

The results are saved (and pickled) in dictionaries for easy lookup.

## Imports

In [72]:
import os
import pickle
import time
import requests 
import json
import datetime

import numpy as np
import pandas as pd

from astral import Location

## Data

In [3]:
cwd = os.getcwd()
data = os.path.join(cwd,'data','wheat-2013-supervised.csv')
df_2013 = pd.read_csv(data)
data = os.path.join(cwd,'data','wheat-2014-supervised.csv')
df_2014 = pd.read_csv(data)


In [4]:
df_2013['Date'].max()

'6/3/2014 0:00'

In [5]:
df_2014['Date'].max()

'6/3/2015 0:00'

## Locations

In [48]:
df_2013['Location'] = list(zip(df_2013['Longitude'], df_2013['Latitude']))
locs_2013 = df_2013['Location'].unique().tolist()
len(locs_2013)

1014

In [49]:
df_2014['Location'] = list(zip(df_2014['Longitude'], df_2014['Latitude']))
locs_2014 = df_2014['Location'].unique().tolist()
len(locs_2014)

1035

In [55]:
# Union of locations without repetitions
elevation = {}
for loc in locs_2013:
    elevation[loc] = 0
for loc in locs_2014:
    if loc in elevation:
        pass
    else:
        elevation[loc] = 0

In [70]:
print('Number of unique locations across 2013 and 2014: {}'.format(len(elevation.keys())))

Number of unique locations across 2013 and 2014: 1167


## Elevation

In [16]:
# Get API key from system variable
google_api_key = os.environ['GOOGLE_API_KEY']

In [71]:
base_string = 'https://maps.googleapis.com/maps/api/elevation/json?locations='
locs = elevation.keys()


for idx, loc in enumerate(locs):
    lat_string = str(loc[1])
    lng_string = str(loc[0])
    call_string = base_string + lat_string + ',' + lng_string + '&key=' + google_api_key
    if idx%10 == 0: 
        print(idx)
    if elevation[loc] == 0:
        response = requests.get(call_string)
        try:
            tmp_elevation = response.json()['results'][0]['elevation']
            elevation[loc] = tmp_elevation
        except:
            print('Something went wrong', lat_string, lng_string)
            print(response.json())

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160


## Length of day

Use the 'Astral' package (https://pythonhosted.org/astral/index.html) to calculate the length of the day (~ hours of sunlight) for each location.

In [73]:
locs = elevation.keys()
length_of_day = {}
for idx, loc in enumerate(locs):
    # Initialize location
    l = Location()
    # Set attributes
    l.name = ''
    l.region = ''
    l.latitude = loc[1]
    l.longitude = loc[0]
    l.timezone = 'UTC'
    l.elevation = elevation[loc]
    sun = l.sun(date=datetime.date(2014, 3, 3),local=True)
    td = sun['sunset'] - sun['sunrise']
    tmp_lod = td.total_seconds() / 3600. 
    length_of_day[loc] = tmp_lod 


819
11.4277777778
462
11.4866666667
235
11.6072222222
259
11.5161111111
945
11.4602777778
140
11.5108333333
811
11.1883333333
1033
11.4761111111
72
11.6155555556
562
11.4286111111
503
11.5022222222
256
11.5075
360
11.44
738
11.4875
505
11.5325
445
11.4122222222
417
11.5188888889
719
11.4197222222
367
11.52
303
11.4130555556
952
11.5172222222
1299
11.1944444444
181
11.5291666667
251
11.4358333333
522
11.5319444444
143
11.5388888889
350
11.4075
242
11.4458333333
296
11.3775
332
11.3919444444
309
11.395
786
11.4147222222
413
11.4480555556
172
11.5377777778
1006
11.4619444444
1146
11.1527777778
709
11.4669444444
1008
11.5019444444
668
11.1336111111
151
11.6158333333
1136
11.1891666667
1189
11.5066666667
1343
11.2030555556
223
11.6055555556
636
11.4191666667
675
11.5197222222
580
11.3666666667
331
11.4855555556
907
11.1263888889
292
11.4344444444
331
11.3958333333
1292
11.4586111111
492
11.4833333333
946
11.4219444444
96
11.5422222222
348
11.4416666667
117
11.5216666667
256
11.5630555556
37

## Save data

In [75]:
# Store data (serialize)
with open(os.path.join('data','elevation.pickle'), 'wb') as handle:
    pickle.dump(elevation, handle)

with open(os.path.join('data','length_of_day.pickle'), 'wb') as handle:
    pickle.dump(length_of_day, handle)



In [76]:
# # Load data (deserialize)
# with open(os.path.join('data','elevation.pickle'), 'rb') as handle:
#     elevation = pickle.load(handle)

